# Karabük 2025 Orman Yangınları: Etki Analizi ve İzleme Raporu

**Hazırlayan:** Yusuf Talha ARABACI
**Kurum:** Karabük Üniversitesi
**Tarih:** 08 Aralık 2025

---

## 1. Yönetici Özeti
Bu rapor, 2025 yaz sezonunda Karabük ilinde meydana gelen orman yangınlarının çevresel etkilerini, yanan alanların büyüklüğünü ve hasar derecesini bilimsel yöntemlerle ortaya koymaktadır. Analiz, **Sentinel-2** uydu görüntüleri ve **Google Earth Engine (GEE)** platformu kullanılarak Karabük Üniversitesi (KBÜ) standartlarında gerçekleştirilmiştir.

### Öne Çıkan Yöntemler
*   **Hassas Maskeleme:** Tarım arazileri ve yerleşim yerleri **ESA WorldCover 2021** verisi kullanılarak analizden çıkarılmış, sadece orman ve çalılık alanlara odaklanılmıştır. Bu sayede hasat edilmiş tarlaların "yanmış" gibi görünmesi engellenmiştir.
*   **İndeksler:** Analizde **dNDVI** (Vejetasyon Değişimi) ve **dNBR** (Yanmışlık Şiddeti Farkı) indeksleri temel alınmıştır. Karmaşık ve işlem yükü yüksek olan RBR/Severity sınıflandırmaları yerine, doğrudan fiziksel değişime odaklanan bu metrikler kullanılmıştır.


## 2. Metodoloji

Çalışmada kullanılan yöntemler uluslararası uzaktan algılama standartlarına (USGS, UN-SPIDER) uygundur.

### 2.1. Veri Kaynakları
*   **Uydu:** Sentinel-2 (MSI Sensörü) - 10m/20m Çözünürlük
*   **Referans (Öncesi) Dönem:** 01 Temmuz - 20 Temmuz 2025 (Yangın öncesi en sağlıklı bitki örtüsü)
*   **Analiz (Sonrası) Dönem:** 05 Eylül - 30 Eylül 2025 (Yangın ve duman etkisinin bittiği dönem)

### 2.2. Kullanılan İndeksler
Yangın etkisini belirlemek için aşağıdaki spektral indeksler kullanılmıştır:

1.  **dNDVI (Delta NDVI):** Bitki örtüsündeki yeşillik kaybını gösterir. Negatif değerler kayıba işaret eder.
2.  **dNBR (Delta NBR):** Yanmış alanları ve toprağın maruz kaldığı ısı stresini gösterir. Yüksek pozitif değerler şiddetli yanmayı ifade eder.

### 2.3. Sınıflandırma ve Maskeleme
*   **Arazi Örtüsü Maskesi:** **ESA WorldCover v200** verisi kullanılarak sadece **"Tree Cover" (Sınıf 10)** ve **"Shrubland" (Sınıf 20)** pikselleri analize dahil edilmiştir. **Tarım arazileri (Cropland) tamamen maskelenmiştir.**


In [ ]:
# --- KURULUM ve AYARLAR ---
import os
import json
import ee
import pandas as pd
from IPython.display import IFrame, display, HTML, Image

# Proje Modülleri
from gee.pipeline import run_pipeline
from gee.utils import ee_init
from gee.aoi import get_aoi

MY_PROJECT = 'solar-bolt-480312-h7'

try:
    ee_init(project=MY_PROJECT)
    print(f"✅ Google Earth Engine bağlantısı başarılı. Proje: {MY_PROJECT}")
except Exception as e:
    print(f"❌ Bağlantı Hatası: {e}")

❌ Bağlantı Hatası: Earth Engine init failed for project 'karabuk-orman-yangini-2025': Caller does not have required permission to use project karabuk-orman-yangini-2025. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=karabuk-orman-yangini-2025 and then retry. Propagation of the new permission may take a few minutes.


In [2]:
# --- PARAMETRELER ---
pre_start = '2025-07-01'
pre_end   = '2025-07-20'
post_start = '2025-09-05'
post_end   = '2025-09-30'

base_out_dir = 'results'
print(f"📅 Analiz Dönemi: {pre_start}___{pre_end} (Öncesi) <---> {post_start}___{post_end} (Sonrası)")

📅 Analiz Dönemi: 2025-07-01___2025-07-20 (Öncesi) <---> 2025-09-05___2025-09-30 (Sonrası)


# 3. İl Geneli Tarama (Overview)


## 3.1. İl Geneli Tarama (Overview)
**Kapsam:** Tüm Karabük İli Sınırları

Bu aşamada tüm Karabük ili taranarak 2025 yaz sezonundaki tüm yangın kümelenmeleri (Aladağ, Ovacık, Eflani hatları) genel ölçekte gözlemlenir.


In [3]:
print("🔄 İl Geneli Analiz Başlatılıyor...")

province_out_dir = os.path.join(base_out_dir, "il_geneli")
province_bbox_path = "karabuk_province_bbox.geojson"

try:
    # İl sınırlarını al
    full_province_geom = get_aoi("KARABUK_PROVINCE")
    
    # Geçici bbox dosyası oluştur
    with open(province_bbox_path, "w", encoding="utf-8") as f:
        json.dump({"type": "Feature", "properties": {"name": "BBOX"}, "geometry": full_province_geom.bounds().getInfo()}, f)

    # PİPELİNE ÇALIŞTIR
    province_outputs = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=province_bbox_path,
        out_dir=province_out_dir,
        area_scale=200,     # Geniş alan için performans optimizasyonu
        min_patch_ha=5,     # 5 hektardan küçük lekeleri yoksay
        skip_severity=False,
        overlay_boundary=full_province_geom
    )
    print("✅ İl analizi tamamlandı.")
    
    # Sadece dNBR göster (Genel Bakış için en iyisi)
    if 'dnbr_map' in province_outputs:
       display(IFrame(src=province_outputs['dnbr_map'], width='100%', height=600))
        
except Exception as e:
    print(f"❌ HATA: {e}")
finally:
    if os.path.exists(province_bbox_path): os.remove(province_bbox_path)


🔄 İl Geneli Analiz Başlatılıyor...
❌ HATA: Caller does not have required permission to use project karabuk-orman-yangini-2025. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=karabuk-orman-yangini-2025 and then retry. Propagation of the new permission may take a few minutes.


# 4. Bölgesel Yangın Analizleri
Yüksek çözünürlüklü (10m) Sentinel-2 görüntüleri ile belirlenen 7 kritik bölge detaylı olarak incelenmiştir. Aşağıdaki analizler, **Karabük 2025 Yangın Sezonu** verilerine dayanmaktadır.

## 4.1. Cildikısık & Kayı (Güney Merkez)
**Konum:** Merkez - Burunsuz & Kayı Köyü

Merkez ilçe güneyinde, Burunsuz ve Kayı köyleri arasındaki ormanlık alanda tespit edilen yangın izleri.

*   **Analiz Yarıçapı:** 3.0 km
*   **Merkez:** `41.063 N, 32.659 E`


In [4]:
zone_name = "1_Cildikisik_Kayi"
center_coords = [32.659, 41.063]
radius_m = 3000

print(f"📍 Analiz Başlıyor: {zone_name} (Yarıçap: {radius_m}m)...")

zone_out_dir = os.path.join(base_out_dir, "yanginlar", zone_name)
center_geom = ee.Geometry.Point(center_coords)
zone_bound = center_geom.buffer(radius_m).bounds()

# GeoJSON Kaydet (Debugging için)
temp_f = f"temp_{zone_name}.geojson"
with open(temp_f, "w") as f:
    json.dump({"type": "Feature", "geometry": zone_bound.getInfo()}, f)

try:
    res = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=temp_f,
        out_dir=zone_out_dir,
        area_scale=10,
        skip_severity=False,
        overlay_boundary=zone_bound
    )
    
    # Çıktıları Göster
    print(f"✅ {zone_name} Analizi Tamamlandı.")
    
    if "dndvi_png" in res:
        display(HTML(f"<b>dNDVI (Bitki Kaybı)</b>"))
        display(Image(filename=res['dndvi_png'], width=600))
        
    if "dnbr_png" in res:
        display(HTML(f"<b>dNBR (Yanmışlık Şiddeti)</b>"))
        display(Image(filename=res['dnbr_png'], width=600))
        
    if "post_rgb_png" in res:
        display(HTML(f"<b>Sonrası RGB (Gerçek Görüntü)</b>"))
        display(Image(filename=res['post_rgb_png'], width=600))

except Exception as e:
    print(f"❌ HATA {zone_name}: {e}")
finally:
    if os.path.exists(temp_f): os.remove(temp_f)


📍 Analiz Başlıyor: 1_Cildikisik_Kayi (Yarıçap: 3000m)...


EEException: Caller does not have required permission to use project karabuk-orman-yangini-2025. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=karabuk-orman-yangini-2025 and then retry. Propagation of the new permission may take a few minutes.

## 4.2. Büyük Ovacık Yangını (En Yıkıcı Yangın)
**Konum:** Ovacık/Safranbolu - Beydini & Alınca Çevresi

**Tarih:** 23-29 Temmuz 2025
**Etki:** Sezonun en yıkıcı yangınıdır. Yaklaşık 5.500 - 6.000 hektar alan etkilenmiş, 15'ten fazla köy tahliye edilmiştir. Yüksek şiddetli (dNBR > 0.44) yanma ve yoğun karaçam/meşe kaybı gözlemlenmiştir. Analiz alanı yangının büyüklüğü nedeniyle 10 km yarıçapında tutulmuştur.

*   **Analiz Yarıçapı:** 10.0 km
*   **Merkez:** `41.098 N, 32.771 E`


In [ ]:
zone_name = "2_Buyuk_Ovacik"
center_coords = [32.771, 41.098]
radius_m = 10000

print(f"📍 Analiz Başlıyor: {zone_name} (Yarıçap: {radius_m}m)...")

zone_out_dir = os.path.join(base_out_dir, "yanginlar", zone_name)
center_geom = ee.Geometry.Point(center_coords)
zone_bound = center_geom.buffer(radius_m).bounds()

# GeoJSON Kaydet (Debugging için)
temp_f = f"temp_{zone_name}.geojson"
with open(temp_f, "w") as f:
    json.dump({"type": "Feature", "geometry": zone_bound.getInfo()}, f)

try:
    res = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=temp_f,
        out_dir=zone_out_dir,
        area_scale=10,
        skip_severity=False,
        overlay_boundary=zone_bound
    )
    
    # Çıktıları Göster
    print(f"✅ {zone_name} Analizi Tamamlandı.")
    
    if "dndvi_png" in res:
        display(HTML(f"<b>dNDVI (Bitki Kaybı)</b>"))
        display(Image(filename=res['dndvi_png'], width=600))
        
    if "dnbr_png" in res:
        display(HTML(f"<b>dNBR (Yanmışlık Şiddeti)</b>"))
        display(Image(filename=res['dnbr_png'], width=600))
        
    if "post_rgb_png" in res:
        display(HTML(f"<b>Sonrası RGB (Gerçek Görüntü)</b>"))
        display(Image(filename=res['post_rgb_png'], width=600))

except Exception as e:
    print(f"❌ HATA {zone_name}: {e}")
finally:
    if os.path.exists(temp_f): os.remove(temp_f)


## 4.3. Kışla Yangını (Ovacık Doğusu)
**Konum:** Ovacık - Kışla Köyü Pazaryeri Mah.

**Tarih:** 23-26 Temmuz 2025
**Etki:** Büyük Ovacık yangınından bağımsız veya sıçrama yoluyla başlayan, yaklaşık 300-500 hektar alanı etkileyen yangın. 10 köyün tahliyesine neden olmuş, orta şiddette (dNBR 0.27-0.44) hasar bırakmıştır.

*   **Analiz Yarıçapı:** 3.0 km
*   **Merkez:** `41.023 N, 32.994 E`


In [ ]:
zone_name = "3_Kisla"
center_coords = [32.994, 41.023]
radius_m = 3000

print(f"📍 Analiz Başlıyor: {zone_name} (Yarıçap: {radius_m}m)...")

zone_out_dir = os.path.join(base_out_dir, "yanginlar", zone_name)
center_geom = ee.Geometry.Point(center_coords)
zone_bound = center_geom.buffer(radius_m).bounds()

# GeoJSON Kaydet (Debugging için)
temp_f = f"temp_{zone_name}.geojson"
with open(temp_f, "w") as f:
    json.dump({"type": "Feature", "geometry": zone_bound.getInfo()}, f)

try:
    res = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=temp_f,
        out_dir=zone_out_dir,
        area_scale=10,
        skip_severity=False,
        overlay_boundary=zone_bound
    )
    
    # Çıktıları Göster
    print(f"✅ {zone_name} Analizi Tamamlandı.")
    
    if "dndvi_png" in res:
        display(HTML(f"<b>dNDVI (Bitki Kaybı)</b>"))
        display(Image(filename=res['dndvi_png'], width=600))
        
    if "dnbr_png" in res:
        display(HTML(f"<b>dNBR (Yanmışlık Şiddeti)</b>"))
        display(Image(filename=res['dnbr_png'], width=600))
        
    if "post_rgb_png" in res:
        display(HTML(f"<b>Sonrası RGB (Gerçek Görüntü)</b>"))
        display(Image(filename=res['post_rgb_png'], width=600))

except Exception as e:
    print(f"❌ HATA {zone_name}: {e}")
finally:
    if os.path.exists(temp_f): os.remove(temp_f)


## 4.4. Toprakcuma Sınır Yangını
**Konum:** Safranbolu Toprakcuma - Kastamonu Sınırı

**Tarih:** 1-3 Eylül 2025
**Etki:** Kastamonu sınırına sıçrayan, yaklaşık 500-800 hektar alanı etkileyen yangın. Araç Karacalar mevkiine doğru yayılım göstermiş, orta-yüksek şiddette hasar oluşturmuştur.

*   **Analiz Yarıçapı:** 3.0 km
*   **Merkez:** `41.189 N, 32.941 E`


In [ ]:
zone_name = "4_Toprakcuma"
center_coords = [32.941, 41.189]
radius_m = 3000

print(f"📍 Analiz Başlıyor: {zone_name} (Yarıçap: {radius_m}m)...")

zone_out_dir = os.path.join(base_out_dir, "yanginlar", zone_name)
center_geom = ee.Geometry.Point(center_coords)
zone_bound = center_geom.buffer(radius_m).bounds()

# GeoJSON Kaydet (Debugging için)
temp_f = f"temp_{zone_name}.geojson"
with open(temp_f, "w") as f:
    json.dump({"type": "Feature", "geometry": zone_bound.getInfo()}, f)

try:
    res = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=temp_f,
        out_dir=zone_out_dir,
        area_scale=10,
        skip_severity=False,
        overlay_boundary=zone_bound
    )
    
    # Çıktıları Göster
    print(f"✅ {zone_name} Analizi Tamamlandı.")
    
    if "dndvi_png" in res:
        display(HTML(f"<b>dNDVI (Bitki Kaybı)</b>"))
        display(Image(filename=res['dndvi_png'], width=600))
        
    if "dnbr_png" in res:
        display(HTML(f"<b>dNBR (Yanmışlık Şiddeti)</b>"))
        display(Image(filename=res['dnbr_png'], width=600))
        
    if "post_rgb_png" in res:
        display(HTML(f"<b>Sonrası RGB (Gerçek Görüntü)</b>"))
        display(Image(filename=res['post_rgb_png'], width=600))

except Exception as e:
    print(f"❌ HATA {zone_name}: {e}")
finally:
    if os.path.exists(temp_f): os.remove(temp_f)


## 4.5. Eflani - Güzelce Yangını
**Konum:** Eflani Saraycık + Araç Güzelce Köyü

**Tarih:** 31 Ağustos - 3 Eylül 2025
**Etki:** Karabük-Kastamonu sınırını aşan en büyük kuzey yangını; duman Ankara'ya kadar ulaştı, 9 köy tahliye edildi.

*   **Analiz Yarıçapı:** 6.0 km
*   **Merkez:** `41.28 N, 32.96 E`


In [ ]:
zone_name = "5_Eflani_Guzelce"
center_coords = [32.96, 41.28]
radius_m = 6000

print(f"📍 Analiz Başlıyor: {zone_name} (Yarıçap: {radius_m}m)...")

zone_out_dir = os.path.join(base_out_dir, "yanginlar", zone_name)
center_geom = ee.Geometry.Point(center_coords)
zone_bound = center_geom.buffer(radius_m).bounds()

# GeoJSON Kaydet (Debugging için)
temp_f = f"temp_{zone_name}.geojson"
with open(temp_f, "w") as f:
    json.dump({"type": "Feature", "geometry": zone_bound.getInfo()}, f)

try:
    res = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=temp_f,
        out_dir=zone_out_dir,
        area_scale=10,
        skip_severity=False,
        overlay_boundary=zone_bound
    )
    
    # Çıktıları Göster
    print(f"✅ {zone_name} Analizi Tamamlandı.")
    
    if "dndvi_png" in res:
        display(HTML(f"<b>dNDVI (Bitki Kaybı)</b>"))
        display(Image(filename=res['dndvi_png'], width=600))
        
    if "dnbr_png" in res:
        display(HTML(f"<b>dNBR (Yanmışlık Şiddeti)</b>"))
        display(Image(filename=res['dnbr_png'], width=600))
        
    if "post_rgb_png" in res:
        display(HTML(f"<b>Sonrası RGB (Gerçek Görüntü)</b>"))
        display(Image(filename=res['post_rgb_png'], width=600))

except Exception as e:
    print(f"❌ HATA {zone_name}: {e}")
finally:
    if os.path.exists(temp_f): os.remove(temp_f)


## 4.6. Aladağ Yangını (Merkez Batı)
**Konum:** Merkez - Aladağ (Kahyalar/Saitler Mevkii)

**Tarih:** 28 Temmuz & 2 Eylül 2025
**Etki:** Merkez ilçe batısında, yaklaşık 700-1.000 hektar çam/meşe ormanını etkileyen yangın. 3 mahalle tahliye edilmiş, rüzgar etkisiyle hızla yayılmıştır. Erozyon riski yüksek bölgelerdendir.

*   **Analiz Yarıçapı:** 3.0 km
*   **Merkez:** `41.127 N, 32.586 E`


In [ ]:
zone_name = "6_Aladag"
center_coords = [32.586, 41.127]
radius_m = 3000

print(f"📍 Analiz Başlıyor: {zone_name} (Yarıçap: {radius_m}m)...")

zone_out_dir = os.path.join(base_out_dir, "yanginlar", zone_name)
center_geom = ee.Geometry.Point(center_coords)
zone_bound = center_geom.buffer(radius_m).bounds()

# GeoJSON Kaydet (Debugging için)
temp_f = f"temp_{zone_name}.geojson"
with open(temp_f, "w") as f:
    json.dump({"type": "Feature", "geometry": zone_bound.getInfo()}, f)

try:
    res = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=temp_f,
        out_dir=zone_out_dir,
        area_scale=10,
        skip_severity=False,
        overlay_boundary=zone_bound
    )
    
    # Çıktıları Göster
    print(f"✅ {zone_name} Analizi Tamamlandı.")
    
    if "dndvi_png" in res:
        display(HTML(f"<b>dNDVI (Bitki Kaybı)</b>"))
        display(Image(filename=res['dndvi_png'], width=600))
        
    if "dnbr_png" in res:
        display(HTML(f"<b>dNBR (Yanmışlık Şiddeti)</b>"))
        display(Image(filename=res['dnbr_png'], width=600))
        
    if "post_rgb_png" in res:
        display(HTML(f"<b>Sonrası RGB (Gerçek Görüntü)</b>"))
        display(Image(filename=res['post_rgb_png'], width=600))

except Exception as e:
    print(f"❌ HATA {zone_name}: {e}")
finally:
    if os.path.exists(temp_f): os.remove(temp_f)


## 4.7. Soğuksu - Arıcak Yangını
**Konum:** Merkez Soğuksu TOKİ + Arıcak Köyü

**Tarih:** 7-10 Ağustos 2025
**Etki:** Şehir merkezine ve TOKİ konutlarına yakınlığıyla büyük tehlike yaratan yangın. Yaklaşık 400-600 hektar alan etkilenmiş, Yeşilköy tahliye edilmiştir.

*   **Analiz Yarıçapı:** 3.0 km
*   **Merkez:** `41.185 N, 32.579 E`


In [ ]:
zone_name = "7_Soguksu_Kayacik"
center_coords = [32.579, 41.185]
radius_m = 3000

print(f"📍 Analiz Başlıyor: {zone_name} (Yarıçap: {radius_m}m)...")

zone_out_dir = os.path.join(base_out_dir, "yanginlar", zone_name)
center_geom = ee.Geometry.Point(center_coords)
zone_bound = center_geom.buffer(radius_m).bounds()

# GeoJSON Kaydet (Debugging için)
temp_f = f"temp_{zone_name}.geojson"
with open(temp_f, "w") as f:
    json.dump({"type": "Feature", "geometry": zone_bound.getInfo()}, f)

try:
    res = run_pipeline(
        pre_start=pre_start, pre_end=pre_end,
        post_start=post_start, post_end=post_end,
        aoi_geojson=temp_f,
        out_dir=zone_out_dir,
        area_scale=10,
        skip_severity=False,
        overlay_boundary=zone_bound
    )
    
    # Çıktıları Göster
    print(f"✅ {zone_name} Analizi Tamamlandı.")
    
    if "dndvi_png" in res:
        display(HTML(f"<b>dNDVI (Bitki Kaybı)</b>"))
        display(Image(filename=res['dndvi_png'], width=600))
        
    if "dnbr_png" in res:
        display(HTML(f"<b>dNBR (Yanmışlık Şiddeti)</b>"))
        display(Image(filename=res['dnbr_png'], width=600))
        
    if "post_rgb_png" in res:
        display(HTML(f"<b>Sonrası RGB (Gerçek Görüntü)</b>"))
        display(Image(filename=res['post_rgb_png'], width=600))

except Exception as e:
    print(f"❌ HATA {zone_name}: {e}")
finally:
    if os.path.exists(temp_f): os.remove(temp_f)
